In [1]:
#-*- coding: utf-8 -*-
#import sys 
#reload(sys)
#sys.setdefaultencoding('utf-8')

import requests
#from pprint import pprint


In [2]:
# kakao api
APP_KEY = "${API_KEY}"
kakao_url = "https://dapi.kakao.com/v2/local/search/address.json"
kakao_headers = {"Authorization" : "KakaoAK {}".format(APP_KEY)}

# naver api
client_id = "${CLIENT_IT}"
client_secret = "${CLIENT_SECRET}"
naver_url = "https://openapi.naver.com/v1/map/geocode" 
naver_headers = {"X-Naver-Client-Id" : "{}".format(client_id) , "X-Naver-Client-Secret": "{}".format(client_secret)}

In [3]:
def kakao_req(params, method):
#     print('HTTP Method: %s' % method)
#     print('Request URL: %s' % kakao_url)
#     print('Headers: %s' % kakao_headers)
#     print('QueryString: %s' % params)

    if method == 'GET':
        return requests.get(kakao_url, headers=kakao_headers, params=params)
    
def naver_req(params, method):
#     print('HTTP Method: %s' % method)
#     print('Request URL: %s' % naver_url)
#     print('Headers: %s' % naver_headers)
#     print('QueryString: %s' % params)
    
    if method == 'GET':
        return requests.get(naver_url, headers=naver_headers, params=params)

In [4]:
def get_location(address):

    params = {"query": "{}".format(address)}

    response = naver_req(params, 'GET')
        

#    #kakao parsing
#     pprint(data['meta']['total_count'])
#     pprint(data["documents"][0]["x"])
#     pprint(data["documents"][0]["y"])
    
        
    return response

In [5]:
import findspark
findspark.init()
import pyspark
import os
import numpy as np

In [6]:
#sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [7]:
basic_folder = '/home/project/real_estate/data/apartment_trade_data/'

file_list = os.listdir(basic_folder)
path_list=[]
for file in file_list:
    print(file)
    path_list.append(basic_folder + file)
    
print(path_list)

apartment_trade_201009.csv
apartment_trade_200801.csv
apartment_trade_201301.csv
apartment_trade_200802.csv
apartment_trade_201707.csv
apartment_trade_201001.csv
apartment_trade_201106.csv
apartment_trade_201612.csv
apartment_trade_201605.csv
apartment_trade_201109.csv
apartment_trade_201805.csv
apartment_trade_201711.csv
apartment_trade_201102.csv
apartment_trade_200901.csv
apartment_trade_201101.csv
apartment_trade_200805.csv
apartment_trade_201610.csv
apartment_trade_201312.csv
apartment_trade_201803.csv
apartment_trade_201002.csv
apartment_trade_201404.csv
apartment_trade_201201.csv
apartment_trade_201108.csv
apartment_trade_200912.csv
apartment_trade_201703.csv
apartment_trade_201202.csv
apartment_trade_201203.csv
apartment_trade_201004.csv
apartment_trade_201209.csv
apartment_trade_201407.csv
apartment_trade_201011.csv
apartment_trade_200909.csv
apartment_trade_201509.csv
apartment_trade_201306.csv
apartment_trade_201409.csv
apartment_trade_201208.csv
apartment_trade_201710.csv
a

In [8]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("header", "true").option("inferSchema", "true").load(path_list)

In [9]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- build_year: integer (nullable = true)
 |-- trade_price: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- city: string (nullable = true)
 |-- sub_city: string (nullable = true)
 |-- dong: string (nullable = true)
 |-- code: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- exclusive_private_area: double (nullable = true)
 |-- address: string (nullable = true)
 |-- detailed_address: string (nullable = true)



In [10]:
df.show()
df.count()

+---+----------+-----------+----+-----+-----+-----+--------+-----+-----+---------------+-----+----------------------+---------------+----------------+
|_c0|build_year|trade_price|year|month|  day| city|sub_city| dong| code|           name|floor|exclusive_private_area|        address|detailed_address|
+---+----------+-----------+----+-----+-----+-----+--------+-----+-----+---------------+-----+----------------------+---------------+----------------+
|  0|      2000|     89,000|2015|    3|11~20|서울특별시|     종로구|  청운동|11110|           청운현대|    1|                129.76| 서울특별시 종로구  청운동|           56-45|
|  1|      2002|     48,500|2015|    3| 1~10|서울특별시|     종로구|  신교동|11110|       신현(101동)|    3|                 84.82| 서울특별시 종로구  신교동|            6-13|
|  2|      2008|     85,000|2015|    3| 1~10|서울특별시|     종로구|  사직동|11110|광화문풍림스페이스본(9-0)|    9|                 94.51| 서울특별시 종로구  사직동|               9|
|  3|      2008|    107,000|2015|    3|11~20|서울특별시|     종로구|  사직동|11110|광화문풍림스페이스본(9-1)|    6|

5967446

In [11]:
#filter_df = df.filter((df["detailed_address"] == "") | df["detailed_address"].isNull() | isnan(df["detailed_address"]))
#filter_df.show()
#filter_df.count() == 36

filter_df = df.na.drop()



In [12]:
from pyspark.sql.functions import *

In [13]:
unique_date_df = filter_df.groupBy(filter_df.year,filter_df.month, filter_df.day).agg(count("day").alias('count'))

unique_date_list = unique_date_df.collect()

new_date_list = []

for unique_date in unique_date_list:
    if unique_date[1] < 10 :
        if int(unique_date[2].split('~')[0]) < 10 : 
            date = str(unique_date[0]) + '-' + '0' + str(unique_date[1]) + '-' + '0' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
        else:
            date = str(unique_date[0]) + '-' + '0' + str(unique_date[1]) + '-' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
        
    else:
        if int(unique_date[2].split('~')[0]) < 10 : 
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' + '0' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
    
        else:
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
    
    
    new_date_list.append([unique_date[0], unique_date[1],unique_date[2],date])
    
    
unique_date_df = sc.parallelize(new_date_list).toDF(['year','month','day','date'])

In [14]:
added_date_df = filter_df.join(unique_date_df, ['year','month','day']).sort('date')
added_date_df.show()
added_date_df.count()

+----+-----+----+---+----------+-----------+-----+--------+------+-----+--------+-----+----------------------+---------------+----------------+-------------------+
|year|month| day|_c0|build_year|trade_price| city|sub_city|  dong| code|    name|floor|exclusive_private_area|        address|detailed_address|               date|
+----+-----+----+---+----------+-----------+-----+--------+------+-----+--------+-----+----------------------+---------------+----------------+-------------------+
|2008|    1|1~10|  9|      2004|     45,000|서울특별시|      중구|   신당동|11140|   삼성홈타운|    6|                104.98|  서울특별시 중구  신당동|             849|2008-01-01T00:00:00|
|2008|    1|1~10|  0|      1981|      4,800|서울특별시|      중구| 장충동2가|11140|      장충|    6|                 31.24|서울특별시 중구  장충동2가|          193-29|2008-01-01T00:00:00|
|2008|    1|1~10|  8|      2002|     37,300|서울특별시|      중구|   신당동|11140|    남산타운|   14|                 59.94|  서울특별시 중구  신당동|             844|2008-01-01T00:00:00|
|2008|    1|1~10

5967410

In [15]:
new_df = added_date_df.groupBy(added_date_df.address,added_date_df.detailed_address).agg(count("detailed_address").alias("count"))
new_df.show()
new_df.count()

+--------------------+----------------+-----+
|             address|detailed_address|count|
+--------------------+----------------+-----+
|     서울특별시 동대문구  장안동|           329-2|  518|
|      서울특별시 노원구  중계동|             600|   26|
|       부산광역시 북구  화명동|            2264| 1659|
|      부산광역시 사상구  학장동|             819|  428|
|       인천광역시 남구  관교동|            13-9|  213|
|      인천광역시 부평구  삼산동|           453-1|  873|
|      인천광역시 부평구  부개동|           477-2|  153|
|      인천광역시 계양구  병방동|             383|  202|
|   울산광역시 울주군 범서읍 구영리|           881-2|  237|
|       경기도 의정부시  민락동|           692-1|  485|
|        경기도 평택시  합정동|             835|  845|
|    경기도 안산시 단원구  초지동|             741| 1543|
|   경기도 고양시 일산서구  주엽동|              90|  294|
|   충청북도 청주시 상당구  금천동|            50-3|  448|
|전라북도 전주시 완산구  중화산동1가|             329|  187|
|       전라북도 익산시  팔봉동|             304|  745|
|       전라북도 익산시  부송동|            1038|  372|
|    전라북도 무주군 무주읍 읍내리|          1216-3|   45|
|        경상북도 안동시  옥동|            

38638

In [16]:
sum = new_df.select(sum("count"))

In [17]:
sum.show()

+----------+
|sum(count)|
+----------+
|   5967410|
+----------+



In [18]:
unique_address_list = new_df.collect()
print unique_address_list[0][0]

서울특별시 동대문구  이문동


In [19]:
location_list = []

# checking why no result data
# response = get_location("경상북도 영천시 신령면 완전리") -> 신녕면
# print response.status_code
# data = response.json()
# print data

index = 1.0;
length = len(unique_address_list)
count = 0;

print length 

for address in unique_address_list:    
    full_address = (address[0] + " " + address[1]).encode('utf8')
    #print str("full " + full_address)
    response = get_location(full_address)

    if response.status_code == 200 :
        
        data = response.json()
        
        
        x = data['result']['items'][0]['point']['x']
        y = data['result']['items'][0]['point']['y']
    
        location_list.append([address[0],address[1],y,x])
        
    else: 
        print str("error address: " + full_address)
        count += 1
        print count
        
        location_list.append([address[0],address[1] ,np.nan,np.nan])
    
    if(index % 100 == 0):
        print ("%0.2f" % (index / length * 100.0)) + "% completed"
        
    index += 1.0

     

8189
1.22% completed
2.44% completed
3.66% completed
4.88% completed
6.11% completed
7.33% completed
8.55% completed
9.77% completed
10.99% completed
12.21% completed
error address: 인천광역시 부평구  부평동 180-24
1
13.43% completed
14.65% completed
15.87% completed
17.10% completed
18.32% completed
19.54% completed
20.76% completed
21.98% completed
23.20% completed
24.42% completed
25.64% completed
26.87% completed
28.09% completed
29.31% completed
30.53% completed
31.75% completed
32.97% completed
34.19% completed
35.41% completed
36.63% completed
37.86% completed
39.08% completed
40.30% completed
41.52% completed
42.74% completed
43.96% completed
45.18% completed
46.40% completed
47.62% completed
48.85% completed
50.07% completed
51.29% completed
52.51% completed
53.73% completed
54.95% completed
56.17% completed
57.39% completed
58.62% completed
59.84% completed
61.06% completed
62.28% completed
63.50% completed
64.72% completed
65.94% completed
67.16% completed
68.38% completed
69.61% compl

In [20]:
#print location_list[0]


#checking_df = df.filter(df.address == "인천광역시 서구  왕길동").filter(df.detailed_address == "가-81-1")

#checking_df.show()
#print(checking_df.count())

                
location = sc.parallelize(location_list).toDF(['address','detailed_address','latitude','longitude'])
location.show()

+-----------------+----------------+----------+-----------+
|          address|detailed_address|  latitude|  longitude|
+-----------------+----------------+----------+-----------+
|  서울특별시 동대문구  이문동|          325-20|37.5944677|127.0593227|
|  서울특별시 서대문구  창천동|           29-81| 37.556057|126.9382413|
|   부산광역시 금정구  구서동|          192-11|35.2542948|129.0893518|
|   대전광역시 유성구  지족동|           863-2|36.3825628|127.3205555|
|  경기도 안산시 상록구  사동|          1197-4|37.3017055|126.8511012|
|부산광역시 기장군 기장읍 대라리|           500-6|35.2424389|129.2117922|
|  서울특별시 중구  의주로1가|            32-2|37.5643082| 126.968413|
|  경기도 평택시 안중읍 안중리|          252-33|36.9881887|126.9255043|
|경상남도 창원시 성산구  상남동|            11-6| 35.222296|128.6851114|
|      경기도 부천시  상동|           449-1|37.4899962|126.7565083|
|   서울특별시 강북구  수유동|            48-1|37.6344091|127.0228738|
|   인천광역시 남동구  만수동|             867|37.4603807|126.7289466|
|  경기도 화성시 남양읍 남양리|          1265-3|37.2101135|126.8181706|
|   서울특별시 강동구  암사동|           502-3|37.5

In [21]:
location.count()

8189

In [22]:
# error : solution -> add duplicate columns(address, detailed address) to join
#new_df_id = new_df.withColumn('id', monotonically_increasing_id())
#new_df_id.show()
#new_df_id.count()
#new_df_id.filter(new_df_id.id == '10000').show()
#add_df.count()


#location_id = location.withColumn('id', row_number().over (Window.partitionBy().partitionBy(location['logitude']).orderBy(lit(1))))
#location_id.show()
#location_id.count()

#location_id.filter(location_id.id == '37777').show()


#add_df = new_df_id.select('address', 'detailed_address','id').join(location_id , new_df_id.id == location_id.id).drop(new_df_id.id)
#add_df.orderBy('id', ascending=True).show()
#add_df.count()

In [23]:
joined_df = added_date_df.join(location.select('address','detailed_address','latitude','longitude'), ['address','detailed_address'])

In [24]:
joined_df.show()
joined_df.count()

+----------------+----------------+----+-----+-----+---+----------+-----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|         address|detailed_address|year|month|  day|_c0|build_year|trade_price|city|sub_city|dong| code|        name|floor|exclusive_private_area|               date|  latitude|  longitude|
+----------------+----------------+----+-----+-----+---+----------+-----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|     경기도 부천시  상동|           449-1|2011|    4| 1~10| 99|      1996|      9,800| 경기도|     부천시|  상동|41190|     동아하이츠빌딩|    8|                 62.74|2011-04-01T00:00:00|37.4899962|126.7565083|
|     경기도 부천시  상동|           449-1|2012|    9| 1~10| 31|      1996|      9,000| 경기도|     부천시|  상동|41190|     동아하이츠빌딩|    9|                 62.74|2012-09-01T00:00:00|37.4899962|126.7565083|
|     경기도 부천시  상동|           449-1|2015|    3|21~3

349140

In [25]:
drop_index_df = joined_df.drop("_c0")
drop_index_df.show()
drop_index_df.count()

+----------------+----------------+----+-----+-----+----------+-----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|         address|detailed_address|year|month|  day|build_year|trade_price|city|sub_city|dong| code|        name|floor|exclusive_private_area|               date|  latitude|  longitude|
+----------------+----------------+----+-----+-----+----------+-----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|     경기도 부천시  상동|           449-1|2011|    4| 1~10|      1996|      9,800| 경기도|     부천시|  상동|41190|     동아하이츠빌딩|    8|                 62.74|2011-04-01T00:00:00|37.4899962|126.7565083|
|     경기도 부천시  상동|           449-1|2012|    9| 1~10|      1996|      9,000| 경기도|     부천시|  상동|41190|     동아하이츠빌딩|    9|                 62.74|2012-09-01T00:00:00|37.4899962|126.7565083|
|     경기도 부천시  상동|           449-1|2015|    3|21~31|      1996|      6

349140

In [27]:
final_apartment_df = drop_index_df.na.drop()
final_apartment_df = final_apartment_df.withColumn('trade_price',regexp_replace('trade_price',"\\,", ""))
final_apartment_df.show()
final_apartment_df.count()

+----------------+----------------+----+-----+-----+----------+-----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|         address|detailed_address|year|month|  day|build_year|trade_price|city|sub_city|dong| code|        name|floor|exclusive_private_area|               date|  latitude|  longitude|
+----------------+----------------+----+-----+-----+----------+-----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|     경기도 부천시  상동|           449-1|2011|    4| 1~10|      1996|       9800| 경기도|     부천시|  상동|41190|     동아하이츠빌딩|    8|                 62.74|2011-04-01T00:00:00|37.4899962|126.7565083|
|     경기도 부천시  상동|           449-1|2012|    9| 1~10|      1996|       9000| 경기도|     부천시|  상동|41190|     동아하이츠빌딩|    9|                 62.74|2012-09-01T00:00:00|37.4899962|126.7565083|
|     경기도 부천시  상동|           449-1|2015|    3|21~31|      1996|       

349114

In [32]:
final_apartment_df.write.csv("final_apartment_trade")

AnalysisException: u'path file:/home/project/final_office_trade already exists.;'